In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import numpy as np

Загружаем данные и смотрим каких значений не хватает.

In [2]:
well=pd.read_csv('..\\well_details.csv')
well.head(3)

,Well_Number,Completion_Type,Top_Perforation,Bottom_Perforation,Depth,Casing_Diam
0,AGB001,VERTICAL PRE-PERFORATED (SWELLABLE PACKERS),495.65,777.44,832.04,6.336
1,AGB002,VERTICAL FRAC,547.19,815.69,847.00,4.892
2,AGB003,VERTICAL FRAC,491.41,800.61,825.00,4.892


In [3]:
well.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231 entries, 0 to 1230
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Well_Number         1231 non-null   object 
 1   Completion_Type     1231 non-null   object 
 2   Top_Perforation     1214 non-null   float64
 3   Bottom_Perforation  1214 non-null   float64
 4   Depth               1205 non-null   float64
 5   Casing_Diam         1216 non-null   float64
dtypes: float64(4), object(2)
memory usage: 57.8+ KB


Исходя из логики геологии(что вряд ли схожие по расположению и глубине скважины имеют большое 
                          различее в расположении пластов, а так же возможные ошибки в данных,
                         в частности неправдоподобно малые глубины скважин), 
заполняем пропущенные значения при помощи KNNinputer из SkLeran. 

In [4]:
well.loc[well['Depth']<100,'Depth']=np.nan

In [5]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)

In [6]:
W=well[['Top_Perforation','Bottom_Perforation','Depth','Casing_Diam']]
W_fill=pd.DataFrame(imputer.fit_transform(W))
well_prop_full=well[['Well_Number','Completion_Type']].join(W_fill.rename({0: 'Top_Perforation',1:'Bottom_Perforation',2:'Depth',3:'Casing_Diam'}, axis=1))
well_prop_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231 entries, 0 to 1230
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Well_Number         1231 non-null   object 
 1   Completion_Type     1231 non-null   object 
 2   Top_Perforation     1231 non-null   float64
 3   Bottom_Perforation  1231 non-null   float64
 4   Depth               1231 non-null   float64
 5   Casing_Diam         1231 non-null   float64
dtypes: float64(4), object(2)
memory usage: 57.8+ KB


Сохраняем файл с заполнеными значениями.

In [8]:
well_prop_full.to_csv('well_full.csv',index=False)